# 6D Emulation

###  Load Libaries

In [1]:
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [4]:
import os
import shutil
import numpy as np
import pandas as pd
import xarray as xr
from frozendict import frozendict
import GPy
from GPy import kern
from GPy.kern import Linear, RBF, Matern32, Matern52
from GPy.models import GPRegression
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.model_wrappers import SimpleGaussianProcessModel, GPyModelWrapper
from emukit.core.initial_designs.latin_design import LatinDesign
from emukit.experimental_design.acquisitions import ModelVariance
from emukit.bayesian_optimization.acquisitions import (
    MaxValueEntropySearch,
    ProbabilityOfImprovement,
    ExpectedImprovement,
)
from emukit.core import ParameterSpace, ContinuousParameter
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import imageio as io
from adcircpy.outputs import Maxele
from sithom.plot import plot_defaults, label_subplots
from sithom.time import timeit
from sithom.place import Point
from sithom.misc import in_notebook
from src.models.generation import ImpactSymmetricTC, Holland08
from src.constants import DATA_PATH, FIGURE_PATH, NEW_ORLEANS, NO_BBOX

In [10]:
@np.vectorize
def indices_in_bbox(lon, lat):
    return (
        lon > NO_BBOX.lon[0]
        and lon < NO_BBOX.lon[1]
        and lat > NO_BBOX.lat[0]
        and lat < NO_BBOX.lat[1]
    )

@np.vectorize
def real_func(param, output_direc: str) -> float:
    point = Point(NEW_ORLEANS.lon + position, NEW_ORLEANS.lat)
    if os.path.exists(output_direc):
        shutil.rmtree(output_direc)
    ImpactSymmetricTC(
        point=point,
        output_direc=output_direc,
        symetric_model=Holland08(),
        angle=angle,
    ).run_impact()
    path = os.path.join(output_direc, "maxele.63.nc")
    maxele = Maxele(path, crs="EPSG:4326")
    index_set = 27
    indices = indices_in_bbox(maxele.x, maxele.y)
    return maxele.values[indices][index_set]

@np.vectorize
def fake_func(param, output_direc: str) -> float:
    return 0.0

class Emulation():
    def __init__(
        self,
        seed=0,
        init_num=40,
        active_num=30,
        indices=100, # plotting function
        acqusition_class=ExpectedImprovement,
        loop_class=BayesianOptimizationLoop,
        kernel_class=RBF,
        x1_range=[-90, 90],
        x2_range=[-2, 3.2],
        path="emu_angle_position",
    ) -> None:
        self.seed = seed
        np.random.seed(seed)
        self.indices = indices
        x1_range, x2_range = self.to_gp_scale(np.array(x1_range), np.array(x2_range))
        x1_range = x1_range.tolist()
        x2_range = x2_range.tolist()
        self.ap = ContinuousParameter("a_param", *x1_range)
        self.bp = ContinuousParameter("b_param", *x2_range)
        self.space = ParameterSpace([self.ap, self.bp])
        self.design = LatinDesign(self.space)
        self.init_num = init_num
        self.active_num = active_num
        self.figure_path = os.path.join(FIGURE_PATH, path)
        self.data_path = os.path.join(DATA_PATH, path)
        self.call_number = 0

        for path in [self.figure_path, self.data_path]:
            if not os.path.exists(path):
                os.mkdir(path)

        # run initial data.
        self.init_x_data = self.design.get_samples(self.init_num).astype("float32")
        self.init_y_data = self.func(self.init_x_data)

        # Fit initial GPyRegression
        self.model_gpy = GPRegression(
            self.init_x_data,
            self.init_y_data.reshape(len(self.init_y_data), 1),
            kernel_class(2, 1),
        )
        self.model_gpy.optimize()

        # active_learning - make acquisition file & loop.
        self.model_emukit = GPyModelWrapper(self.model_gpy)
        if acqusition_class is MaxValueEntropySearch:
            self.acquisition_function = acqusition_class(
                model=self.model_emukit, space=self.space
            )
        else:
            self.acquisition_function = acqusition_class(model=self.model_emukit)

        self.loop = loop_class(
            model=self.model_emukit,
            space=self.space,
            acquisition=self.acquisition_function,
            batch_size=1,
        )

        # get ready for active learning.
        self.active_x_data = np.array([[np.nan, np.nan]])
        self.active_y_data = np.array([[np.nan]])

        # make initial plot
        self.plot()
        plt.savefig(os.path.join(self.figure_path, f"0.png"))
        if in_notebook():
            plt.show()
        else:
            plt.clf()

        for i in range(1, active_num + 1):
            print(i)
            self.run_loop(1)
            self.plot()
            plt.savefig(os.path.join(self.figure_path, f"{i}.png"))
            if in_notebook():
                plt.show()
            else:
                plt.clf()

        with io.get_writer(f"{self.figure_path}.gif", mode="I", duration=0.5) as writer:
            for file_name in [
                os.path.join(self.figure_path, f"{i}.png")
                for i in range(self.active_num + 1)
            ]:
                image = io.imread(file_name)
                writer.append_data(image)
        writer.close()
        self.save_data()

    def save_data(self):
        init_x, init_y = self.init_data()
        active_x, active_y = self.active_data()
        ds = xr.Dataset(
            data_vars=dict(
                init_x=(["inum", "var"], init_x),
                init_y=(["inum"], init_y[:, 0]),
                active_x=(["anum", "var"], active_x),
                active_y=("anum", active_y[:, 0]),
            ),
            coords=dict(
                var=(["var"], ["x1", "x2"]),
            ),
            attrs=dict(description="Training Data"),
        )
        file_name = os.path.join(self.data_path, "data.nc")
        if not os.path.exists(file_name):
            ds.to_netcdf(file_name)
        else:
            print("File Already Exists!")

    def run_loop(self, new_iterations):
        self.loop.run_loop(self.func, new_iterations)
        self.active_x_data = self.loop.loop_state.X[len(self.init_x_data) :]
        self.active_y_data = self.loop.loop_state.Y[len(self.init_x_data) :]

    def init_data(self):
        return (
            self.merge(*self.to_real_scale(*self.split(self.init_x_data))),
            -self.init_y_data,
        )

    def active_data(self):
        return (
            self.merge(*self.to_real_scale(*self.split(self.active_x_data))),
            -self.active_y_data,
        )

    def __repr__(self) -> str:
        return f"seed = {self.seed}, init_num = {self.init_num}, active_num = {self.active_num}"

    def to_gp_scale(self, x1, x2):
        return (x1 + 60) / 10, (x2 - 0.6) / 0.5

    def to_real_scale(self, x1, x2):
        return x1 * 10 - 60, x2 / 2 + 0.6

    def merge(self, x1, x2):
        return np.concatenate(
            [x1.reshape(*x1.shape, 1), x2.reshape(*x2.shape, 1)], axis=-1
        )

    def split(self, data):
        shape = data.shape
        if len(shape) == 2:
            return data[:, 0], data[:, 1]
        elif len(shape) == 3:
            return data[:, :, 0], data[:, :, 1]
        else:
            assert False

    def ob_func(self, angle: np.ndarray, position: np.ndarray) -> float:
        # print(angle, position)
        num = len(angle)
        output_direc = [
            os.path.join(self.data_path, str(self.call_number + i)) for i in range(num)
        ]
        self.call_number += num
        return -smash_func(angle, position, output_direc)

    def func(self, data) -> float:
        output = self.ob_func(*self.to_real_scale(*self.split(data)))
        return output.reshape(len(output), 1)

    def learnt_function(self, x1, x2):
        mean, var = self.model_emukit.predict(self.merge(*self.to_gp_scale(x1, x2)))
        return -mean, np.std(var)

    def plot(self) -> None:
        a_indices = np.linspace(self.ap.min, self.ap.max, num=self.indices)
        b_indices = np.linspace(self.bp.min, self.bp.max, num=self.indices)
        a_indices, b_indices = self.to_real_scale(a_indices, b_indices)
        a_mesh, b_mesh = np.meshgrid(a_indices, b_indices)
        length = len(a_indices) * len(b_indices)
        a_array = a_mesh.ravel()
        b_array = b_mesh.ravel()
        comb_array = np.zeros([length, 2])
        comb_array[:, 0] = a_array[:]
        comb_array[:, 1] = b_array[:]
        comb_array_gp = self.merge(*self.to_gp_scale(*self.split(comb_array)))

        # Evaluate Gaussian Process
        mean, var = self.model_emukit.predict(comb_array_gp)
        mean_mesh = -mean[:, 0].reshape(self.indices, self.indices)
        std_mesh = np.sqrt(var[:, 0]).reshape(self.indices, self.indices)
        # Evaluate Acquisition Function
        aq_mesh = self.acquisition_function.evaluate(comb_array_gp).reshape(
            self.indices, self.indices
        )

        # Set up plot
        plot_defaults()
        fig, axs = plt.subplots(
            2, 2, sharex=True, sharey=True
        )
        label_subplots(axs, override="outside")

        ax = axs[0, 1]
        ax.set_title("Acq. Func.")
        im = ax.contourf(a_mesh, b_mesh, aq_mesh)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        fig.colorbar(im, cax=cax, orientation="vertical")

        ax = axs[0, 0]
        init_x, init_y = self.init_data()
        active_x, active_y = self.active_data()
        im = ax.scatter(
            init_x[:, 0],
            init_x[:, 1],
            c=init_y,
            marker="x",
            label="original data points",
        )
        ax.scatter(
            active_x[:, 0],
            active_x[:, 1],
            c=active_y,
            marker="+",
            label="new data points",
        )
        divider = make_axes_locatable(ax)
        ax.set_title("Samples")
        cax = divider.append_axes("right", size="5%", pad=0.05)
        fig.colorbar(im, cax=cax, orientation="vertical")
        ax.set_ylabel("Position [$^{\circ}$E]")

        ax = axs[1, 0]
        ax.set_title("Prediction Mean")
        im = ax.contourf(a_mesh, b_mesh, mean_mesh)  # , vmin=0, vmax=5.6)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        fig.colorbar(im, cax=cax, orientation="vertical")
        ax.set_ylabel("Position [$^{\circ}$E]")
        ax.set_xlabel("Bearing [$^{\circ}$]")
        ax = axs[1, 1]
        ax.set_title("Pred. Std. Dev. ")
        im = ax.contourf(a_mesh, b_mesh, std_mesh)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        fig.colorbar(im, cax=cax, orientation="vertical")
        ax.set_xlabel("Bearing [$^{\circ}$]")


def poi():
    Emulation(
        acqusition_class=ProbabilityOfImprovement, path="emulation_angle_pos_poi"
    )


def poi_long():
    Emulation(
        acqusition_class=ProbabilityOfImprovement,
        path="emulation_angle_pos_poi_long",
        init_num=100,
        active_num=50,
    )


def mves():
    Emulation(
        acqusition_class=MaxValueEntropySearch, path="emulation_angle_pos_mves"
    )


def inp_diff():
    Emulation(
        path="emulation_angle_pos_big",
        seed=20,
        init_num=100,
        active_num=30,
    )




In [34]:
def param(updates):
    defaults = {
        # Trajectory
        "angle": 0.0, # degrees from North
        "speed": 7.71, # m s**-1
        "point_east": 0.6, # degrees East of New Orleans
        # Radial Profile of Tropical Cyclone - Holland Hurricane Parameters
        "rmax": 40744.0, # meters
        "pc": 92800.0, # Pa
        "vmax": 54.01667, # m s**-1
        "xn": 1.1249, # dimensionless
    }
    # no surprises
    assert np.all([x in defaults.keys() for x in updates.keys()])

    output = defaults.copy()

    for key in updates:
        output[key] = updates[key]

    return output


def holliday(updates):
    assert "pc" in updates.keys()
    updates["vmax"] = vmax_from_pressure_holliday(92800)
    return updates

In [37]:
param(holliday({"angle":0.0, "pc": 92900.0}))

{'angle': 0.0,
 'speed': 7.71,
 'point_east': 0.6,
 'rmax': 40744.0,
 'pc': 92900.0,
 'vmax': 54.01667,
 'xn': 1.1249}

In [32]:
from src.models.generation import vmax_from_pressure_holliday

In [33]:
vmax_from_pressure_holliday(92800)

54.01667

In [39]:
angles = ContinuousParameter("angle", -90, 90)
speeds = ContinuousParameter("speed", 2, 14)
point_east = ContinuousParameter("point_east", -0.6, 1.2)
rmax = ContinuousParameter("rmax", 2, 14)
pc = ContinuousParameter("pc", 900, 980)
# vmax = ContinuousParameter("vmax", 20, )
xn = ContinuousParameter("xn", 0.8, 1.4)

space = ParameterSpace([angles, speeds, point_east, rmax, pc, xn])

In [50]:
design = LatinDesign(space)

In [55]:
init_x_data = design.get_samples(300).astype("float32")

In [56]:
init_x_data.shape

(300, 6)

In [47]:
?LatinDesign

In [48]:
?design

In [174]:
def get_param(updates):
    defaults = {
        # Trajectory
        "angle": 0.0,  # degrees from North
        "speed": 7.71,  # m s**-1
        "point_east": 0.6,  # degrees East of New Orleans
        # Radial Profile of Tropical Cyclone - Holland Hurricane Parameters
        "rmax": 40744.0,  # meters
        "pc": 92800.0,  # Pa
        "vmax": 54.01667,  # m s**-1
        "xn": 1.1249,  # dimensionless
    }
    # no surprises
    assert np.all([x in defaults.keys() for x in updates.keys()])

    output = defaults.copy()

    for key in updates:
        output[key] = updates[key]

    return output


def holliday_vmax(updates):
    assert "pc" in updates.keys()
    updates["vmax"] = vmax_from_pressure_holliday(92800)
    return updates


@np.vectorize
def indices_in_bbox(lon, lat):
    return (
        lon > NO_BBOX.lon[0]
        and lon < NO_BBOX.lon[1]
        and lat > NO_BBOX.lat[0]
        and lat < NO_BBOX.lat[1]
    )


def real_func(param, output_direc: str) -> float:
    point = Point(NEW_ORLEANS.lon + param["position_east"], NEW_ORLEANS.lat)
    if os.path.exists(output_direc):
        shutil.rmtree(output_direc)
    ImpactSymmetricTC(
        point=point,
        output_direc=output_direc,
        symetric_model=Holland08(
            param["pc"], param["rmax"], param["vmax"], param["xn"]
        ),
        angle=param["angle"],
        trans_speed=param["speed"],
    ).run_impact()
    path = os.path.join(output_direc, "maxele.63.nc")
    maxele = Maxele(path, crs="EPSG:4326")
    index_set = 27
    indices = indices_in_bbox(maxele.x, maxele.y)
    return maxele.values[indices][index_set]


def fake_func(param, output_direc: str) -> float:
    default_param = get_param({})
    assert np.all([key in default_param.keys() for key in param])
    print("called fake func")
    if os.path.exists(output_direc):
        shutil.rmtree(output_direc)
    return 0.0


class TestFeature:
    """

    Data conversion example::
        >>> import numpy as np
        >>> tf = TestFeature()
        >>> x_data = tf.samples(300)
        >>> np.all(np.isclose(tf.to_real(tf.to_gp(x_data)), x_data, rtol=1e-6))
        True
        >>> np.all(tf.from_param(tf.to_param(x_data[0])) == x_data[0])
        True
    """

    def __init__(self, dryrun=False) -> None:
        self.dryrun = dryrun
        angles = ContinuousParameter("angle", -90, 90)
        speeds = ContinuousParameter("speed", 2, 14)
        point_east = ContinuousParameter("point_east", -0.6, 1.2)
        rmax = ContinuousParameter("rmax", 2, 14)
        pc = ContinuousParameter("pc", 900, 980)
        # vmax = ContinuousParameter("vmax", 20, )
        xn = ContinuousParameter("xn", 0.8, 1.4)
        self.space = ParameterSpace([angles, speeds, point_east, rmax, pc, xn])
        self.names = self.space.parameter_names
        self.ideal_space = ParameterSpace(
            [ContinuousParameter(name, 0, 1) for name in self.names]
        )
        self.design = LatinDesign(self.space)
        bounds = self.space.get_bounds()
        self.lower_bounds = np.asarray(bounds)[:, 0].reshape(1, len(bounds))
        self.upper_bounds = np.asarray(bounds)[:, 1].reshape(1, len(bounds))
        self.diffs = self.upper_bounds - self.lower_bounds
        self.call_number = 0

    def samples(self, num_samples: int) -> np.ndarray:
        return design.get_samples(num_samples).astype("float32")

    def to_real(self, x_data: np.ndarray) -> np.ndarray:
        """
        x_data: assume last dimension is the variables.
        """
        ones = np.ones((x_data.shape[0], 1))
        return np.dot(ones, self.lower_bounds) + x_data * np.dot(ones, self.diffs)

    def to_gp(self, x_data: np.ndarray) -> np.ndarray:
        """
        x_data: assume last dimension is the variables.
        """
        ones = np.ones((x_data.shape[0], 1))
        return (x_data - np.dot(ones, self.lower_bounds)) * np.dot(ones, 1 / self.diffs)

    def to_param(self, x_data_point: np.ndarray) -> dict:
        assert len(x_data_point) == len(self.names)
        assert len(np.shape(x_data_point)) == 1
        return holliday_vmax(
            {self.names[i]: x_data_point[i] for i in range(len(self.names))}
        )

    def from_param(self, param_dict: dict) -> np.ndarray:
        assert np.all([name in param_dict for name in self.names])
        output_np = np.zeros(len(self.names))
        for i, name in enumerate(self.names):
            output_np[i] = param_dict[name]
        return output_np

    def func(self, x_data: np.ndarray) -> np.ndarray:
        real_data = self.to_real(x_data)
        shape = np.shape(real_data)
        output_list = []
        
        for i in range(shape[0]):
            param = self.to_param(real_data[i])
            print("Calling", param)
            output_direc =  str(self.call_number)
            print(output_direc)
            if self.dryrun:
                output_list.append(fake_func(param, output_direc))
            else:
                output_list.append(real_func(param, output_direc))
            self.call_number += 1
            
        return np.array(output_list).reshape(len(output_list),1)

In [175]:
tf = TestFeature(dryrun=True)

In [176]:
tf.func(tf.to_gp(tf.samples(10)))

Calling {'angle': -45.0, 'speed': 3.799999952316284, 'point_east': -0.3300000131130219, 'rmax': 5.0, 'pc': 912.0, 'xn': 1.25, 'vmax': 54.01667}
0
called fake func
Calling {'angle': -9.0, 'speed': 6.199999809265137, 'point_east': 0.7500000000000001, 'rmax': 11.0, 'pc': 952.0, 'xn': 1.0099999904632568, 'vmax': 54.01667}
1
called fake func
Calling {'angle': -81.0, 'speed': 12.199999809265137, 'point_east': 0.5699999928474425, 'rmax': 8.600000381469727, 'pc': 936.0, 'xn': 1.3700000047683716, 'vmax': 54.01667}
2
called fake func
Calling {'angle': 9.000000000000014, 'speed': 13.399999618530273, 'point_east': -0.5099999904632568, 'rmax': 9.800000190734863, 'pc': 968.0, 'xn': 1.1299999952316284, 'vmax': 54.01667}
3
called fake func
Calling {'angle': 81.0, 'speed': 11.0, 'point_east': 0.029999999329447746, 'rmax': 13.399999618530273, 'pc': 960.0, 'xn': 1.309999942779541, 'vmax': 54.01667}
4
called fake func
Calling {'angle': 63.0, 'speed': 8.600000381469727, 'point_east': 1.1100000143051147, 'r

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [118]:
tf.space

In [124]:
np.all(tf.from_param(tf.to_param(init_x_data[0])) == init_x_data[0])

True

In [93]:
tf.to_real(tf.to_gp(init_x_data)).shape

(300, 6)

In [94]:
init_x_data.shape

(300, 6)

In [108]:
tf.space.parameter_names

['angle', 'speed', 'point_east', 'rmax', 'pc', 'xn']

In [114]:
init_x_data[0]

array([ 83.7  ,   9.5  ,   1.161,  10.46 , 909.2  ,   0.949],
      dtype=float32)

In [142]:
?Holland08

In [143]:
?ImpactSymmetricTC

In [ ]:
trans_speed=param["speed"]